# Pràctiques de Nous Usos de la Informàtica

**Nom de les persones del grup:** Mr X & Ms. Z

# Pràctica 2. Recomanadors

# Construcció d'un recomanador.

In [1]:
N=100

In [33]:
# lectura de dades

import pandas as pd
import numpy as np

unames = ['user_id', 'gender', 'age', 'occupation', 'zip']
users = pd.read_table('ml-1m/users.dat', sep='::', header=None, names=unames, engine='python')
rnames = ['user_id', 'movie_id', 'rating', 'timestamp']
ratings = pd.read_table('ml-1m/ratings.dat', sep='::', header=None, names=rnames, engine='python')
mnames = ['movie_id', 'title', 'genres']
movies = pd.read_table('ml-1m/movies.dat', sep='::', header=None, names=mnames, engine='python')

data = pd.merge(pd.merge(ratings, users), movies)
data = data[data.user_id < N]
#data = data[data.movie_id < N]

**Alerta**: Les implementacions dels exercicis 6, 7 i 8 poden tardar molt en executar-se, considera fer-ho en un subset de les dades originals. En la 1a cel·la:
```
    data = data[data.user_id < 100]
    data = data[data.movie_id < 100]
```
El codi anterior limitaria les dades a 100 usuaris i 100 películes. Recorda re-executar les cel·les.

Com a guia, una implementació que usi N usuaris i películes, per l'exercici 6, ot arribar a trigar:

* N=100, 5 segons a 60 segons
* N=1000, 15 minuts a 1 hora
* N=10000, 20 hores a 60 hores

segons la implementació utilitzada

El següent codi, donat el conjunt de dades, construeix un conjunt d'entrenament i un conjunt  de test:

In [34]:
# generem subconjunts de training i test
def assign_to_set(df):
    sampled_ids = np.random.choice(df.index,
                                   size=np.int64(np.ceil(df.index.size * 0.2)),
                                   replace=False)
    df.ix[sampled_ids, 'for_testing'] = True
    return df

data['for_testing'] = False
grouped = data.groupby('user_id', group_keys=False).apply(assign_to_set)
movielens_train = data[grouped.for_testing == False]
movielens_test = data[grouped.for_testing == True]
print movielens_train.shape
print movielens_test.shape

(10281, 11)
(2619, 11)


La següent funció `evaluate(estimate)`, donat un conjunt de dades d'entrenament i un conjunt de dades de test ens avalua la precisió d'un sistema de recomanació que li passem per paràmetre. Per a cadascun dels elements del conjunt de test haurem de pronosticar el seu valor i comparar-lo amb el valor real que l'usuari li ha asignat. La mesura que utilizarem per avaluar el sistema és la root-mean-square error (rsme)

In [4]:
# definim una funció per avaluar el resultat de la recomanació.

def compute_rmse(y_pred, y_true):
    return np.sqrt(np.mean(np.power(y_pred - y_true, 2)))

def evaluate(estimate,test=movielens_test):
    ids_to_estimate = zip(test['user_id'], test['movie_id'])
    estimated = np.array([estimate(u,i) for (u,i) in ids_to_estimate])
    real = test.rating.values
    nans = np.isnan(estimated)
    return compute_rmse(estimated[~nans], real[~nans])

### EXERCICI 4

+ Construeix dues funcions, `dist_euclid(x,y)` i `coef_pearson(x,y)`, que implementin la distància Euclidiana i el coeficient de correlació de Pearson entre dos vectors usant funcions de pandas. 

+ Escriu les funcions que calculin la semblança entre dos series d'un DataFrame de Pandas. S'utiltizaran per calcular les similituds entre usuaris o entre items:

    + ``def sim_euclid (data_frame, row1, row2)``
    Calcula els vectors representatius de cada fila, C1 i C2, amb les puntuacions de les columnes que estan presents en ambdós files. En el cas dels usuaris (files), això implica trobar les películes (columnes) que han puntuat tots dos.<br />Si no hi ha puntuacions en comú, retornar 0. En cas contrari, retornar ``1/(1+dist_euclid(C1, C2))``

    + ``def sim_pearson (data_frame, row1, row2)``
    Calcular els vectors representatius de cada fila, C1 i C2, amb les puntuacions de les columnes que estan presents en ambdós files.<br />Si no hi ha puntuacions en comú, retornar 0. Retornar ``coef_pearson(C1,C2)``
    

In [46]:
import math
import numpy as np
import pandas as pd

# Returns the euclidean distance of two vectors
def dist_euclid(x, y):
    return np.linalg.norm(x-y)

# Returns the Pearson correlation of two vectors 
def coef_pearson(x, y):
    return np.corrcoef(x, y)[0][1]

# Returns a distance-based similarity score for person1 and person2 based on euclidean distance
def sim_euclid(data_frame, row1, row2):
    ind = data_frame.loc[row1].dropna().index.intersection(data_frame.loc[row2].dropna().index)
    if len(ind) == 0:
        return 0
    x = data_frame.loc[row1][ind]
    y = data_frame.loc[row2][ind]
    return 1/(1 + dist_euclid(x, y))

# Returns a distance-based similarity score for person1 and person2 based on pearson distance
def sim_pearson(data_frame, row1, row2):
    ind = data_frame.loc[row1].dropna().index.intersection(data_frame.loc[row2].dropna().index)
    if len(ind) == 0:
        return 0
    x = data_frame.loc[row1, ind]
    y = data_frame.loc[row2, ind]
    return coef_pearson(x, y)

Tests de les funcions, pots realitzar modificacions prèvies a les taules (per exemple, agrupar o pivotar) per accelerar el procés

In [66]:
# Execute functions
new_data = data.pivot("user_id", "movie_id", "rating")
print sim_euclid(new_data, 1, 2)
print sim_pearson(new_data, 1, 2)

0.333333333333
0.416666666667


### EXERCICI 5

+ Feu dues funcions, ``get_best_euclid(data_frame, user, n)`` i ``get_best_pearson(data_frame, user, n)``, que retornin els ``n`` usuaris més semblants segons aquestes dues mesures de similitud.

In [69]:
import heapq

# return the N most similar users to a given user based on euclidean distance
def get_best_euclid(data_frame, user, n):
    new_data = data_frame.pivot("user_id", "movie_id", "rating")
    users = data_frame[data_frame.user_id != user]["user_id"].unique()
    return heapq.nsmallest(n, users, key=lambda x: sim_euclid(new_data, x, user))

# return the N most similar users to a given user based on pearson correlation
def get_best_pearson(data_frame, user, n):
    new_data = data_frame.pivot("user_id", "movie_id", "rating")
    users = data_frame[data_frame.user_id != user]["user_id"].unique()
    return heapq.nlargest(n, users, key=lambda x: sim_pearson(new_data, x, user))

In [70]:
# Execute functions
# Aquestes funcions poden trigar a executar-se; feu proves primer amb una part petita de la base de dades.

print get_best_euclid(data, 1, 5)
print get_best_pearson(data, 1, 5)

[46, 92, 26, 75, 53]
[72, 66, 57, 40, 47]


### EXERCICI 6

En l'exercici 6 i 7 es desenvoluparà un sistema de recomanació basat en usuaris i en ítems, respectivament.

El codi donat, que es basa en 3 classes, és la recomenada per fer-ho òptim i reaprofitar el màxim de codi, però s'acceptaran solucions que no la segueixin, sempre hi quan respectin el mètode "estimate" explicat més abaix i funcionin de forma correcte.

#### `CollaborativeFiltering`

Una classe base, comuna en els 2 recomanadors, que implementarà:
  
  * `__init__`: Rep com a paràmetres el dataframe (que constarà de `user_id`, `movie_id` i `rating`), la funció de semblança (Euclidiana o Pearson) que volem usar i un paràmetre `M` que indica el tamany que tindrà la matriu de similituds.
  
  * `precompute`: Generar per cada estimació la semblança entre 2 usuaris o items seria molt costós i faria l'algorisme molt lent, per tant, aquesta funció omplirà la taula MxM (on M es el número de usuaris o items, segons el recomanador) amb el coeficient de semblança.
      * Nota: La taula es un DataFrame de Pandas, per tant accedirem als element fent servir l'indexat de Pandas (que correspon al id del user/movie, i no a la posició 0...i)
  
  * `estimate`: s'encarrega de donar la predicció, en aquest cas donat un usuari i una pel·lícula estimar el seu rating.
    + Nota 1: Si un `user_id` o `movie_id` no es troba en el DataFrame, cal retornar "np.NAN"
    + Nota 2: En el recomenador d'usuaris, s'ha d'evitar comparar `user_id` a ell mateix. De la mateixa forma, en el d'items evitarem comparar un `movie_id` amb sí mateix.

#### `UserRecomender`

Recomanador basat en usuaris que hereta de `CollaborativeFiltering`. Implementarà:

  * `__init__`: Pot realitzar transformacions al DataFrame
  
#### `ItemRecomender`

Recomanador basat en items que hereta de `CollaborativeFiltering`. Implementarà:

  * `__init__`: Pot realitzar transformacions al DataFrame
    

In [78]:
class CollaborativeFiltering(object):
    """ Collaborative filtering using a custom sim(u,u'). """
    
    def __init__(self, data, M, similarity=sim_pearson):
        """ Constructor """
        self.sim_method = similarity # Gets recommendations for a person by using a weighted average
        self.df = data 
        self.sim = pd.DataFrame(0, index=self.df.index, columns=self.df.index)
        
    def precompute(self):
        from itertools import combinations
        """Prepare data structures for estimation. Compute similarity matrix self.sim"""
        for i,j in combinations(self.sim.index, 2):
            self.sim.loc[i, j] = self.sim.loc[j, i] = self.sim_method(self.df, i, j)
        print self.sim
    def estimate(self, row, col):
        """ Given an row (user_id in 6, movie_id in 7) and a column (movie_id in 6, user_id in 7) 
            returns the estimated rating """
        if row not in self.df.index or col not in self.df.columns:
            return np.nan
        similarities = self.sim.loc[row]
        def f(score, value):
            if score < 0:
                return (5 - value) * -score
            return value * score
        tmp = [f(sim_score, self.df.loc[id_value, col])
               for id_value, sim_score in similarities.iteritems()
               if not np.isnan(self.df.loc[id_value, col])
               if not np.isnan(sim_score)]
        #print tmp
        total = sum(tmp)
        return total / similarities.apply(abs).sum()
        

In [79]:
class UserRecomender(CollaborativeFiltering):
    """ Recomender using Collaborative filtering with a User similarity (u,u'). """
    
    def __init__(self, data_train, similarity=sim_pearson):
        """ Constructor """
        
        # You should do any transformation to data_train (grouping/pivot/...) here, if needed
        transformed_data = data_train.pivot("user_id", "movie_id", "rating")
        
        super(UserRecomender, self).__init__(transformed_data, data_train.user_id.unique(), similarity)

                
    def estimate(self, user_id, movie_id):
        """ Given an user_id and a movie_id returns the estimated rating for such movie """
        return super(UserRecomender, self).estimate(user_id, movie_id)
        

In [80]:
user_reco = UserRecomender(movielens_train)
user_reco.precompute()
user_reco.estimate(user_id=2, movie_id=1)

user_id        1         2         3         4         5         6         7   \
user_id                                                                         
1        0.000000 -0.577350 -0.522233  0.500000 -0.098533  0.868599       NaN   
2       -0.577350  0.000000 -0.250000 -0.288675 -0.421637 -0.389249  0.147832   
3       -0.522233 -0.250000  0.000000  0.774597 -0.333333 -0.333333  0.763763   
4        0.500000 -0.288675  0.774597  0.000000       NaN       NaN -0.188982   
5       -0.098533 -0.421637 -0.333333       NaN  0.000000 -0.681385 -0.138343   
6        0.868599 -0.389249 -0.333333       NaN -0.681385  0.000000       NaN   
7             NaN  0.147832  0.763763 -0.188982 -0.138343       NaN  0.000000   
8       -0.750000 -0.010747 -0.422577  0.866025  0.282860 -0.438529  0.211702   
9        0.140563  0.554274 -0.522233       NaN -0.045727 -0.339683  0.698430   
10       0.011450 -0.322031 -0.005318 -0.519291 -0.112818  0.122810  0.487950   
11      -0.158114  0.000000 

0.62616342438188044

In [81]:
evaluate(user_reco.estimate, movielens_test)

3.5041755770742564

### EXERCICI 7

+ Desenvolupa un sistema de recomanació col·laboratiu basat en ítems. Si la classe `CollaborativeFiltering` s'ha fet prou genèrica, tan sols caldrà fer petites modificacions a `__init__`, del contrari, podeu fer les modificacions que cregueu necessàries.

In [ ]:
class ItemRecomender(CollaborativeFiltering):
    """ Recomender using Collaborative filtering with a Item similarity (i,i'). """
    
    def __init__(self,data_train, similarity=sim_pearson):
        """ Constructor """
        
        # You should do any transformation to data_train (grouping/pivot/...) here, if needed
        transformed_data = data_train
        
        super(ItemRecomender, self).__init__(transformed_data, data_train.movie_id.unique(), similarity)

            
    def estimate(self, user_id, movie_id):
        """ Given an user_id and a movie_id returns the estimated rating for such movie """
        return super(ItemRecomender, self).estimate(movie_id, user_id)
        

In [ ]:
item_reco = ItemRecomender(movielens_train)
item_reco.precompute()
item_reco.estimate(user_id=2, movie_id=1)

In [ ]:
evaluate(item_reco.estimate, movielens_test)

### EXERCICI 8

* Feu un nou mètode `get_recomendations(user_id, n)` que retorni les n pel·lícules recomenades per a l'usuari user_id. De nou, és recomenable fer-ho a la clase pare, `CollaborativeFiltering`, cridant-la des dels fills de forma semblant a com fa `estimate`.

* Executeu la funció en els dos recomenadors 

In [ ]:
user_reco.get_recomendations(1, 5)

In [ ]:
item_reco.get_recomendations(1, 5)